In [1]:
#I have referenced the below links for some of the code. It is a web page containing spark documentation
# https://spark.apache.org/docs/2.4.5/ml-features.html

In [2]:
from pyspark.sql import SparkSession, Row
import numpy as np
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import Tokenizer,RegexTokenizer

In [3]:
from pyspark.sql.functions import isnan, when, count, col

In [4]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [5]:
sc = spark.sparkContext

In [6]:
#cookbook text files
cb_txt = sc.wholeTextFiles('dbfs:/FileStore/tables/*.txt')

In [7]:
#notice that the file name and the contents are in tuple pair
cb_txt.take(2)

Out[62]: [('dbfs:/FileStore/tables/amem.txt',
 ' \n \n\n \n \n\n \n \n \n\n\n The American Matron: Or, Practical and Scientific Cookery. \n\n\n By a Housekeeper. \n\n\n Boston: J. Munroe &amp; Co., 1851 \n\n [Page images for The American Matron were produced before MSU began the "Feeding America" digitization project. This title is not formally part of the project and will not be transcribed.] \n\n \n\n\n\n \n\n \n\n\n \n\n'),
 ('dbfs:/FileStore/tables/amwh.txt',
 ' \r\n \r\n \r\n The American Woman\'s Home: or, Principles of Domestic Science; being a Guide to the Formation and Maintenance of Economical, Healthful, Beautiful, and Christian Homes. Beecher, Catharine Esther Stowe, Harriet Beecher Home economics. Introduction. The Christian Family. A Christian House. A Healthful Home. Scientific Domestic Ventilation. Stoves, Furnaces, and Chimneys. Home Decoration. The Care of Health. Domestic Exercise. Healthful Food. Healthful Drinks. Cleanliness. Clothing. Good Cooking. Early Rising. Domestic Manners. Good Temper in the Housekeeper. Habits of System and Order. Giving in Charity. Economy of Time and Expenses. Health of Mind. The Care of Infants. The Management of Young Children. Domestic Amusements and Social Duties. Care of the Aged. The Care of Servants. Care of the Sick. Accidents and Antidotes. Sewing, Cutting, and Mending. Fires and Lights. The Care of Rooms. The Care of Yards and Gardens. The Propagation of Plants. The Cultivation of Fruit. The Care of Domestic Animals. Earth-Closets. Warming and Ventilation. Care of the Homeless, the Helpless, and the Vicious. The Christian Neighborhood. Appeal to American Women. Glossary of Words and References. Analytical Index. New York, J. B. Ford and Company; Boston, H. A. Brown &amp; Co. Electronic edition created by Digital &amp; Multimedia Center, Michigan State University Libraries, East Lansing, Michigan, 2002-2003. Supplementary material by Jan Longone, Anne-Marie Rachman, Peter Berg, Yvonne Lockwood, and Val Berryman 1869 Text xml-external-parsed-entity gif quicktime http://digital.lib.msu.edu/cookbooks/americanwomanshome/amwh.xml--&gt; OCLC 7875017 en Digitized as part of "Feeding America: The Historic American Cookbook Project." Michigan State University Libraries, East Lansing, Michigan, 2002-2003. http://digital.lib.msu.edu/cookbooks/ United States Nineteenth century The book digitized here was published in the United States before 1923 and is in the public domain according to U.S. copyright law. The digital version and supplementary materials are made available for all educational uses. \r\n \r\n \r\n \r\n A scene depicting a family reading together in a library. A woman sits at a table with a young girl on her lap, holding a large book for the girl to read, while a man and a young boy look on from beside and behind her, respectively. An older man sitting in a chair further off has also closed the book in his lap and is watching the scene. A second young girl is sitting on the floor in the foreground, holding a doll and looking intently at some toys that are scattered on the floor at her feet. \r\n THE Two young boys stand near a woman in a rocking chair, who is holding a baby on her lap. One of the boys is holding out a kitten to the baby, who is stretching out her hand toward it. An adult cat standing at the woman\'s feet looks up at the scene curiously. AMERICAN WOMAN\'S An illustration of a beautiful home in the countryside, with a small path leading up to the front door. A man and a woman stand on the path, looking at the house; another couple are standing on the lawn some distance from the house, gazing out over a small river that flows behind the house. HOME BY CATHARINE BEECHER AND HARRIET BEECHER STOWE An illustration of two women and a young girl in a garden. One of the women is holding a tiny parasol, while the other stoops to pick some flowers. The young girl is holding up a basket full of already picked flowers. NEW YORK J. B. FORD &amp; CO. 1869. \r\n \r\n 

In [8]:
cb_txt.count()

Out[63]: 76

In [9]:
#remove the file name from rdd as it is not required
cb_txt2 = cb_txt.map(lambda line : (( line[1])))

In [10]:
#replace \n from the content
cb_txt2 = cb_txt2.map(lambda r: r.replace('\n', '')) 

In [11]:
cb_txt2.take(1)

Out[100]: [' The American Matron: Or, Practical and Scientific Cookery. By a Housekeeper. Boston: J. Munroe &amp; Co., 1851 [Page images for The American Matron were produced before MSU began the "Feeding America" digitization project. This title is not formally part of the project and will not be transcribed.] ']

In [12]:
cb_txt2 = cb_txt2.map(lambda r: Row(r))

In [13]:
#convert to dataframe
cb_txt2 = cb_txt2.toDF()

In [14]:
cb_txt2.count()

Out[103]: 76

In [15]:
cb_txt2.show()

+--------------------+
 _1|
+--------------------+
 The Amer...|
    The Ameri...|
     Manual F...|
    "Aunt Bab...|
         ...|
      ...|
         ...|
        ...|
     Breakfas...|
       Pr...|
    Cooking ...|
     Dr. Cha...|
     Chinese...|
     Chocolat...|
     Common S...|
     The Com...|
     The Cook...|
     La Cuisi...|
     Directio...|
     Dishes ...|
+--------------------+
only showing top 20 rows

In [16]:
#Rename column 
cb_txt2 = cb_txt2.withColumnRenamed('_1','cookbook_text')

In [17]:
#trim column for leading and trailing white spaces
from pyspark.sql.functions import trim
cb_txt2 = cb_txt2.withColumn("cookbook_text", trim(cb_txt2.cookbook_text))

In [18]:
cb_txt2.show()

+--------------------+
 cookbook_text|
+--------------------+
The American Matr...|
   The Americ...|
    Manual Fo...|
   "Aunt Babe...|
        ...|
     ...|
        ...|
        ...|
    Breakfast...|
      Pra...|
   Cooking i...|
    Dr. Chas...|
    Chinese-...|
    Chocolate...|
    Common Se...|
    The Comp...|
    The Cook'...|
    La Cuisin...|
    Direction...|
    Dishes &...|
+--------------------+
only showing top 20 rows

In [19]:
#tokenize the cookbook_text column
retoken = RegexTokenizer(inputCol="cookbook_text", outputCol="words", pattern="\\W")
cb_csv3 = retoken.transform(cb_txt2)

In [20]:
cb_csv3.show()

+--------------------+--------------------+
 cookbook_text| words|
+--------------------+--------------------+
The American Matr...|[the, american, m...|
   The Americ...|[the, american, w...|
    Manual Fo...|[manual, for, arm...|
   "Aunt Babe...|[aunt, babette, s...|
        ...|[the, ideal, bart...|
     ...|[a, bookplate, il...|
        ...|[the, blue, grass...|
        ...|[the, boston, coo...|
    Breakfast...|[breakfast, lunch...|
      Pra...|[practical, house...|
   Cooking i...|[cooking, in, old...|
    Dr. Chas...|[dr, chase, s, re...|
    Chinese-...|[chinese, japanes...|
    Chocolate...|[chocolate, and, ...|
    Common Se...|[common, sense, i...|
    The Comp...|[the, complete, c...|
    The Cook'...|[the, cook, s, ow...|
    La Cuisin...|[la, cuisine, cre...|
    Direction...|[directions, for,...|
    Dishes &...|[dishes, amp, bev...|
+--------------------+--------------------+
only showing top 20 rows

In [21]:
#With shingles of size 5 we now create a count vectorizer
# This count vectorizer will extract vocabulary and generate a CountVectorizerModel
#this is the first step of 3 for duplicate MinHash
vec = CountVectorizer(inputCol = 'words', outputCol = 'vectors', vocabSize = 5, minDF = 2.0)
shingles_cv = vec.fit(cb_csv3).transform(cb_csv3)

In [22]:
shingles_cv.show()

+--------------------+--------------------+--------------------+---+
 cookbook_text| words| vectors| id|
+--------------------+--------------------+--------------------+---+
The American Matr...|[the, american, m...|(5,[0,1,2,3],[4.0...| 0|
   The Americ...|[the, american, w...|(5,[0,1,2,3,4],[1...| 1|
    Manual Fo...|[manual, for, arm...|(5,[0,1,2,3,4],[4...| 2|
   "Aunt Babe...|[aunt, babette, s...|(5,[0,1,2,3,4],[5...| 3|
        ...|[the, ideal, bart...|(5,[0,1,2,3,4],[2...| 4|
     ...|[a, bookplate, il...|(5,[0,1,2,3,4],[5...| 5|
        ...|[the, blue, grass...|(5,[0,1,2,3,4],[1...| 6|
        ...|[the, boston, coo...|(5,[0,1,2,3,4],[2...| 7|
    Breakfast...|[breakfast, lunch...|(5,[0,1,2,3,4],[6...| 8|
      Pra...|[practical, house...|(5,[0,1,2,3,4],[9...| 9|
   Cooking i...|[cooking, in, old...|(5,[0,1,2,3,4],[1...| 10|
    Dr. Chas...|[dr, chase, s, re...|(5,[0,1,2,3,4],[9...| 11|
    Chinese-...|[chinese, japanes...|(5,[0,1,2,3,4],[6...| 12|
    Chocolate...|[chocolate, and, ...|(5,[0,1,2,3,4],[1...| 13|
    Common Se...|[common, sense, i...|(5,[0,1,2,3,4],[8...| 14|
    The Comp...|[the, complete, c...|(5,[0,1,2,3,4],[5...| 15|
    The Cook'...|[the, cook, s, ow...|(5,[0,1,2,3,4],[1...| 16|
    La Cuisin...|[la, cuisine, cre...|(5,[0,1,2,3,4],[3...| 17|
    Direction...|[directions, for,...|(5,[0,1,2,3,4],[7...| 18|
    Dishes &...|[dishes, amp, bev...|(5,[0,1,2,3,4],[2...| 19|
+--------------------+--------------------+--------------------+---+
only showing top 20 rows

In [23]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + unique id
#note some ids are big numbers as monotonically_increasing_id has not autoincremented but all id values are unique so it should'nt be a problem 
shingles_cv = shingles_cv.withColumn("id", monotonically_increasing_id())

In [24]:
shingles_cv.select('id').show(76)

+----------+
 id|
+----------+
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
 9|
 10|
 11|
 12|
 13|
 14|
 15|
 16|
 17|
 18|
 19|
 20|
 21|
 22|
 23|
 24|
 25|
 26|
 27|
 28|
 29|
 30|
 31|
 32|
 33|
 34|
 35|
 36|
 37|
 38|
 39|
 40|
 41|
 42|
 43|
 44|
 45|
 46|
 47|
 48|
 49|
8589934592|
8589934593|
8589934594|
8589934595|
8589934596|
8589934597|
8589934598|
8589934599|
8589934600|
8589934601|
8589934602|
8589934603|
8589934604|
8589934605|
8589934606|
8589934607|
8589934608|
8589934609|
8589934610|
8589934611|
8589934612|
8589934613|
8589934614|
8589934615|
8589934616|
8589934617|
+----------+

In [25]:
#applying minhash on shingles_cv
min_hash = MinHashLSH(inputCol="vectors", outputCol="Hashes", seed = 12345)
min_hash2 = min_hash.fit(shingles_cv)
min_hash2.transform(shingles_cv).head()

Out[123]: Row(cookbook_text='The American Matron: Or, Practical and Scientific Cookery. By a Housekeeper. Boston: J. Munroe &amp; Co., 1851 [Page images for The American Matron were produced before MSU began the "Feeding America" digitization project. This title is not formally part of the project and will not be transcribed.]', words=['the', 'american', 'matron', 'or', 'practical', 'and', 'scientific', 'cookery', 'by', 'a', 'housekeeper', 'boston', 'j', 'munroe', 'amp', 'co', '1851', 'page', 'images', 'for', 'the', 'american', 'matron', 'were', 'produced', 'before', 'msu', 'began', 'the', 'feeding', 'america', 'digitization', 'project', 'this', 'title', 'is', 'not', 'formally', 'part', 'of', 'the', 'project', 'and', 'will', 'not', 'be', 'transcribed'], vectors=SparseVector(5, {0: 4.0, 1: 2.0, 2: 1.0, 3: 1.0}), id=0, Hashes=[DenseVector([133824602.0])])

In [26]:
#I have created minhash3 for the sole purpose of showing the column with hashs after minHash
minhash3 = min_hash.fit(shingles_cv).transform(shingles_cv)

In [27]:
minhash3.show()

+--------------------+--------------------+--------------------+---+----------------+
 cookbook_text| words| vectors| id| Hashes|
+--------------------+--------------------+--------------------+---+----------------+
The American Matr...|[the, american, m...|(5,[0,1,2,3],[4.0...| 0|[[1.33824602E8]]|
   The Americ...|[the, american, w...|(5,[0,1,2,3,4],[1...| 1|[[1.33824602E8]]|
    Manual Fo...|[manual, for, arm...|(5,[0,1,2,3,4],[4...| 2|[[1.33824602E8]]|
   "Aunt Babe...|[aunt, babette, s...|(5,[0,1,2,3,4],[5...| 3|[[1.33824602E8]]|
        ...|[the, ideal, bart...|(5,[0,1,2,3,4],[2...| 4|[[1.33824602E8]]|
     ...|[a, bookplate, il...|(5,[0,1,2,3,4],[5...| 5|[[1.33824602E8]]|
        ...|[the, blue, grass...|(5,[0,1,2,3,4],[1...| 6|[[1.33824602E8]]|
        ...|[the, boston, coo...|(5,[0,1,2,3,4],[2...| 7|[[1.33824602E8]]|
    Breakfast...|[breakfast, lunch...|(5,[0,1,2,3,4],[6...| 8|[[1.33824602E8]]|
      Pra...|[practical, house...|(5,[0,1,2,3,4],[9...| 9|[[1.33824602E8]]|
   Cooking i...|[cooking, in, old...|(5,[0,1,2,3,4],[1...| 10|[[1.33824602E8]]|
    Dr. Chas...|[dr, chase, s, re...|(5,[0,1,2,3,4],[9...| 11|[[1.33824602E8]]|
    Chinese-...|[chinese, japanes...|(5,[0,1,2,3,4],[6...| 12|[[1.33824602E8]]|
    Chocolate...|[chocolate, and, ...|(5,[0,1,2,3,4],[1...| 13|[[1.33824602E8]]|
    Common Se...|[common, sense, i...|(5,[0,1,2,3,4],[8...| 14|[[1.33824602E8]]|
    The Comp...|[the, complete, c...|(5,[0,1,2,3,4],[5...| 15|[[1.33824602E8]]|
    The Cook'...|[the, cook, s, ow...|(5,[0,1,2,3,4],[1...| 16|[[1.33824602E8]]|
    La Cuisin...|[la, cuisine, cre...|(5,[0,1,2,3,4],[3...| 17|[[1.33824602E8]]|
    Direction...|[directions, for,...|(5,[0,1,2,3,4],[7...| 18|[[1.33824602E8]]|
    Dishes &...|[dishes, amp, bev...|(5,[0,1,2,3,4],[2...| 19|[[1.33824602E8]]|
+--------------------+--------------------+--------------------+---+----------------+
only showing top 20 rows

In [28]:
minhash3.count()

Out[127]: 76

In [29]:
#now we calculate the Jacard Distance. The lower the Jacard Distance the more the 2 txt files are duplicates as it caluculates dissimilarity b/w sample sets. So a Jacard Distance of 0 means that the txt files are complete duplicate Reference: https://en.wikipedia.org/wiki/Jaccard_index

threshold = 0.6
min_hash2.approxSimilarityJoin(shingles_cv, shingles_cv, threshold, distCol="JaccardDistance")\
    .select(col("datasetA.id").alias("idA"),
        col("datasetB.id").alias("idB"),
        col("JaccardDistance")).show()

+---+----------+-------------------+
idA| idB| JaccardDistance|
+---+----------+-------------------+
 6| 0|0.19999999999999996|
 7|8589934611| 0.0|
 8|8589934607| 0.0|
 5| 21| 0.0|
 3| 17| 0.0|
 2| 8| 0.0|
 9| 43| 0.0|
 3|8589934615| 0.0|
 7|8589934616| 0.0|
 9| 6| 0.0|
 5|8589934603| 0.0|
 5| 25| 0.0|
 3|8589934592| 0.0|
 6| 4| 0.0|
 8|8589934602| 0.0|
 10| 45| 0.0|
 10| 28| 0.0|
 0| 6|0.19999999999999996|
 0|8589934601|0.19999999999999996|
 4| 18| 0.0|
+---+----------+-------------------+
only showing top 20 rows